In [5]:
import pandas as pd
import os
import sys
import logging
import pandas as pd
from google.cloud import bigquery
from hashlib import md5
from typing import List

In [ ]:
#defined project, dataset and eventual table name
PROJECT_NAME = "team-week-10"
DATASET_NAME = "nfl_sports_betting"
TABLE_NAME = "dim_stadiums"

#establish schema 

TABLE_METADATA = {
    "stadiums": {
        "table_name": "dim_stadiums",
        "schema": [
            bigquery.SchemaField("stadium_name", "string", mode="REQUIRED"),
            bigquery.SchemaField("stadium_location", "string", mode="REQUIRED"),
            bigquery.SchemaField("stadium_open", "int64", mode="NULLABLE"),
            bigquery.SchemaField("stadium_close", "int64", mode="NULLABLE"),
            bigquery.SchemaField("stadium_type", "string", mode="NULLABLE"),
            bigquery.SchemaField("zipcode", "int64", mode="NULLABLE"),
            bigquery.SchemaField("weather_type", "string", mode="NULLABLE"),
            bigquery.SchemaField("capacity", "int64", mode="NULLABLE"),
            bigquery.SchemaField("surface", "string", mode="NULLABLE"),
            bigquery.SchemaField("elevation", "int64", mode="NULLABLE"),
        ]
    }

}